# HW 2 (due November 30 23:59 MSK)

### Problem 1 (1 points)

Diffie–Hellman key exchange protocol is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. 

1. Implement function to generate common secret key within multiplicative group of given Finite field with known generator 

2. Test your solution in GF(17) with generator g=11. Bobs' open key B=11, Alice private key is a=7

In [1]:
GF, g, B, a = 17, 11, 11, 7

def secret_key(sec_num, mod, GF):
    return mod**sec_num % GF

print(secret_key(GF, B, a))

2


### Problem 2 (3 points)

El Gamal protocol is widely used in cryptography. In this task we will ask you to implement your own El-Gamal encryption scheme on Python

1. Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed within the lectures). (1 point)

In [2]:
import random
import math
import sys

class PrivateKey(object):
    def __init__(self, p=None, g=None, x=None, iNumBits=0):
        self.p = p
        self.g = g
        self.x = x
        self.iNumBits = iNumBits

class PublicKey(object):
    def __init__(self, p=None, g=None, h=None, iNumBits=0):
        self.p = p
        self.g = g
        self.h = h
        self.iNumBits = iNumBits

# the greatest common denominator for a and b(a > b)
def gcd( a, b ):
        while b != 0:
            c = a % b
            a = b
            b = c
        return a

# tests if num is prime
def prime_check( num, iConfidence ):
        for i in range(iConfidence):
                a = random.randint( 1, num-1 )
                if gcd( a, num ) > 1:
                        return False
                if not jacobi( a, num ) % num == pow(a, (num-1)//2, num):
                        return False
        return True

#computes the jacobi symbol of a, n
def jacobi( a, n ):
        if a == 0:
                if n == 1:
                        return 1
                else:
                        return 0
        elif a == -1:
                if n % 2 == 0:
                        return 1
                else:
                        return -1
        elif a == 1:
                return 1
        elif a == 2:
                if n % 8 == 1 or n % 8 == 7:
                        return 1
                elif n % 8 == 3 or n % 8 == 5:
                        return -1
        elif a >= n:
                return jacobi( a%n, n)
        elif a%2 == 0:
                return jacobi(2, n)*jacobi(a//2, n)

        else:
                if a % 4 == 3 and n%4 == 3:
                        return -1 * jacobi( n, a)
                else:
                        return jacobi(n, a )

def find_primitive_root( p ):
        if p == 2:
                return 1
        p1 = 2
        p2 = (p-1) // p1
        while( 1 ):
                g = random.randint( 2, p-1 )
                if not (pow(g, (p-1)//p1, p) == 1):
                        if not pow(g, (p-1)//p2, p) == 1:
                                return g

#find n bit prime
def find_prime(iNumBits, iConfidence):
        while(1):
                p = random.randint( 2**(iNumBits-2), 2**(iNumBits-1) )
                while( p % 2 == 0 ):
                        p = random.randint(2**(iNumBits-2),2**(iNumBits-1))
                        
                while( not prime_check(p, iConfidence) ):
                        p = random.randint( 2**(iNumBits-2), 2**(iNumBits-1) )
                        while( p % 2 == 0 ):
                                p = random.randint(2**(iNumBits-2), 2**(iNumBits-1))
                p = p * 2 + 1 # if p is prime
                if prime_check(p, iConfidence):
                        return p

#encodes bytes to integers mod p.
def encode(sPlaintext, iNumBits):
        byte_array = bytearray(sPlaintext, 'utf-16')
        z = []
        k = iNumBits//8
        j = -1 * k
        num = 0
        for i in range( len(byte_array) ):
                if i % k == 0:
                        j += k
                        num = 0
                        z.append(0)
                z[j//k] += byte_array[i]*(2**(8*(i%k)))
        return z

#decode integers to the original message bytes
def decode(aiPlaintext, iNumBits):
        bytes_array = []
        k = iNumBits//8
        for num in aiPlaintext:
                for i in range(k):
                        temp = num
                        for j in range(i+1, k):

                                temp = temp % (2**(8*j))
                        letter = temp // (2**(8*i))
                        bytes_array.append(letter)
                        num = num - (letter*(2**(8*i)))
        decodedText = bytearray(b for b in bytes_array).decode('utf-16')
        return decodedText

#generate public key K1 (p, g, h) and private key K2 (p, g, x)
def generate_keys(iNumBits=256, iConfidence=32):
        p = find_prime(iNumBits, iConfidence)
        g = find_primitive_root(p)
        g = pow(g, 2, p)
        x = random.randint( 1, (p - 1) // 2 )
        h = pow( g, x, p )
        publicKey = PublicKey(p, g, h, iNumBits)
        privateKey = PrivateKey(p, g, x, iNumBits)
        return {'privateKey': privateKey, 'publicKey': publicKey}

2. Implement functions that realize the encryption and decryption in El Gamal protocol. (1 points)

In [3]:
def encrypt(key, sPlaintext):
        z = encode(sPlaintext, key.iNumBits)
        cipher_pairs = []
        for i in z:
                y = random.randint( 0, key.p )
                c = pow( key.g, y, key.p )
                d = (i*pow( key.h, y, key.p)) % key.p
                cipher_pairs.append( [c, d] )

        encryptedStr = ""
        for pair in cipher_pairs:
                encryptedStr += str(pair[0]) + ' ' + str(pair[1]) + ' '
        return encryptedStr

# decryption on the cipher pairs found in Cipher using
def decrypt(key, cipher):
        plaintext = []
        cipherArray = cipher.split()
        if (not len(cipherArray) % 2 == 0):
            return "Malformed Cipher Text"
        for i in range(0, len(cipherArray), 2):
                c = int(cipherArray[i])
                d = int(cipherArray[i+1])
                s = pow( c, key.x, key.p )
                plain = (d*pow( s, key.p-2, key.p)) % key.p
                plaintext.append( plain )
        decryptedText = decode(plaintext, key.iNumBits)
        decryptedText = "".join([ch for ch in decryptedText if ch != '\x00'])
        
        return decryptedText

3. Calculate Hash of your name by SHA-1 and test El Gamal encryption/decryption functions on it (1 points)

In [4]:
def sha1(data):
    bytes = ""

    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0

    for n in range(len(data)):
        bytes+='{0:08b}'.format(ord(data[n]))
    bits = bytes+"1"
    pBits = bits
    #pad until length equals 448 mod 512
    while len(pBits)%512 != 448:
        pBits+="0"
    pBits+='{0:064b}'.format(len(bits)-1)

    def chunks(l, n):
        return [l[i:i+n] for i in range(0, len(l), n)]

    def rol(n, b):
        return ((n << b) | (n >> (32 - b))) & 0xffffffff

    for c in chunks(pBits, 512): 
        words = chunks(c, 32)
        w = [0]*80
        for n in range(0, 16):
            w[n] = int(words[n], 2)
        for i in range(16, 80):
            w[i] = rol((w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]), 1)  

        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        for i in range(0, 80):
            if 0 <= i <= 19:
                f = (b & c) | ((~b) & d)
                k = 0x5A827999
            elif 20 <= i <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= i <= 59:
                f = (b & c) | (b & d) | (c & d) 
                k = 0x8F1BBCDC
            elif 60 <= i <= 79:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = rol(a, 5) + f + e + k + w[i] & 0xffffffff
            e = d
            d = c
            c = rol(b, 30)
            b = a
            a = temp

        h0 = h0 + a & 0xffffffff
        h1 = h1 + b & 0xffffffff
        h2 = h2 + c & 0xffffffff
        h3 = h3 + d & 0xffffffff
        h4 = h4 + e & 0xffffffff

    return '%08x%08x%08x%08x%08x' % (h0, h1, h2, h3, h4)

In [5]:
keys = generate_keys() # keys generation
priv = keys['privateKey']
pub = keys['publicKey']

message = "Stanislav Krikunov" # enter my name
hash_sha1 = sha1(message) # hash sha1 calculation
cipher = encrypt(pub, hash_sha1) # encode to cipher using public key
decrypted_hash_sha1 = decrypt(priv, cipher) # decode using cipher and private key

In [6]:
print(decrypted_hash_sha1, hash_sha1)
print()
if decrypted_hash_sha1 == hash_sha1:
    print('equal')
else:
    print('not equal')

2a1f0c988399f3c228e5c2387da1bf4b3771e8e0 2a1f0c988399f3c228e5c2387da1bf4b3771e8e0

equal


## Result:
### sha1 hash: 2a1f0c988399f3c228e5c2387da1bf4b3771e8e0
### EL Gamal gives true result

### Problem 3 (3 points)

Elliptic curves due to their efficient hardware realization widely used in modern secure communication channels. The main thing that lies inside their cryptographic hardness is that we can break them only by greed search over all group points. In this task, we will ask you to write python function that returns all group elements of a certain elliptic curve over a finite field 

1. Write a python function that list all points of elliptic curve $y^2=x^3-5x-8$ over $F_{11}$ (2 points)

In [7]:
import collections
Coord = collections.namedtuple("Coordinates", ["x", "y"])

def inverse(n, q):
    for i in range(q):
        if (n * i) % q == 1:
            return i
        pass
    pass

def check(n, q):
    for i in range(1, q):
        if i * i % q == n:
            return (i, q - i)
        pass
    return (-1, -1)

class EC(object):
    def __init__(self, a, b, q):
        self.a = a
        self.b = b
        self.q = q
        self.zero = Coord(0, 0)
        pass

    # finding points on curve at x < q
    def find(self, x):
        ysq = (x ** 3 + self.a * x + self.b) % self.q
        y, my = check(ysq, self.q)
        
        if (y, my) == (-1,-1): # return None if there is no point in the grup
            return None
        else:
            return Coord(x, y), Coord(x, my)
    
     # addition for 2 points
    def addition(self, p1, p2):
        if p1 == self.zero: return p2
        if p2 == self.zero: return p1
        if p1.x == p2.x and (p1.y != p2.y or p1.y == 0):
            return self.zero
        if p1.x == p2.x:
            l = (3 * p1.x * p1.x + self.a) * inverse(2 * p1.y, self.q) % self.q
            pass
        else:
            l = (p2.y - p1.y) * inverse(p2.x - p1.x, self.q) % self.q
            pass
        x = (l * l - p1.x - p2.x) % self.q
        y = (l * (p1.x - x) - p1.y) % self.q
        return Coord(x, y)

In [8]:
a, b, q = -5, -8, 11
ec = EC(a,b,q)

In [9]:
elements = []
print('poins indexes from group')
for i in range(1, 11, 1):
    if ec.find(i) != None:
        elements.append(ec.find(i))
        print(i)
elements

poins indexes from group
2
3
4
5
7
9


[(Coordinates(x=2, y=1), Coordinates(x=2, y=10)),
 (Coordinates(x=3, y=2), Coordinates(x=3, y=9)),
 (Coordinates(x=4, y=5), Coordinates(x=4, y=6)),
 (Coordinates(x=5, y=2), Coordinates(x=5, y=9)),
 (Coordinates(x=7, y=5), Coordinates(x=7, y=6)),
 (Coordinates(x=9, y=4), Coordinates(x=9, y=7))]

2. Compute the sum of two points in group above and make sure that result lies within the same group (1 point)

In [10]:
# take index 4, for example
point1, point2 = ec.find(4)
print(point1, point2)

Coordinates(x=4, y=5) Coordinates(x=4, y=6)


In [11]:
# and index 9
point3, point4 = ec.find(9)
print(point3, point3)

Coordinates(x=9, y=4) Coordinates(x=9, y=4)


In [12]:
# find the sum of 4 and 9 points
g_sum = ec.addition(point1, point4)
print(g_sum)

Coordinates(x=3, y=2)


In [13]:
# make sure that result lies within the same group
ec.find(g_sum.x), ec.find(g_sum.y)

((Coordinates(x=3, y=2), Coordinates(x=3, y=9)),
 (Coordinates(x=2, y=1), Coordinates(x=2, y=10)))

In [14]:
# function to check point
def is_in_group(elements, point):
    in_group = False
    for i in range(len(elements)):
        for j in range(2):
            if elements[i][j] == ec.find(point)[j]:
                in_group = True
    return in_group

In [15]:
is_in_group(elements, g_sum.x)

True

In [16]:
is_in_group(elements, g_sum.y)

True

### Problem 4 (3 points)

Merkle hash trees play an important role in forming transaction blocks in blockchain. In this assignment we ask you to plot your own Merkle hash tree and check its' properties. Below we provide you with some code fragment what you can use in your assignment

In [17]:
import matplotlib
import networkx as nx
%matplotlib qt5
from networkx import balanced_tree, draw_networkx, draw

Let us plot graph basis for Merkle hash tree

In [64]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 4],
    [-2, 3],
    [2, 3],
    [-3, 2],
    [-1, 2],
    [1, 2],
    [3, 2],
    [-3, 1],
    [-1, 1],
    [1, 1],
    [3, 1]
]
parents = [0, 0, 0, 1, 1, 2, 2, 3, 4, 5, 6]
for index in range(11):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: 'tx1',
    8: 'tx2',
    9: 'tx3',
    10: 'tx4',
}
nx.draw_networkx_labels(G, positions, labels = labels)

{0: Text(0, 4, '0'),
 1: Text(-2, 3, '1'),
 2: Text(2, 3, '2'),
 3: Text(-3, 2, '3'),
 4: Text(-1, 2, '4'),
 5: Text(1, 2, '5'),
 6: Text(3, 2, '6'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

In Bitcoin double sha256 hash scheme is used. Here is an example.

In [65]:
import hashlib

first_hash = hashlib.sha256(b"hello") # "b" stands for binary representation
second_hash = hashlib.sha256()
print('First hash represented as a hexadecimal number:', first_hash.hexdigest())
second_hash.update(first_hash.digest())
print('Second hash represented as a hexadecimal number:', second_hash.hexdigest())

First hash represented as a hexadecimal number: 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
Second hash represented as a hexadecimal number: 9595c9df90075148eb06860365df33584b75bff782a510c6cd4883a419833d50


Now we can easily change vertices' labels to hashes of corresponding messages and plot new graph

In [66]:
labels[3] = hashlib.sha256(hashlib.sha256(b"tx1").digest()).hexdigest()

# and plot the graph again

nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)

{0: Text(0, 4, '0'),
 1: Text(-2, 3, '1'),
 2: Text(2, 3, '2'),
 3: Text(-3, 2, '856a4921cd32690244af7568e7bd1391a94119e17c7f33234f4bf11271b223e5'),
 4: Text(-1, 2, '4'),
 5: Text(1, 2, '5'),
 6: Text(3, 2, '6'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

1. Construct Merkle hash tree using previously constructed graph by finding corresponding SHA256 hashes on vertices and plot obtained Merkle hash tree (1 point)

In [67]:
def data_hashing(data_input):
    hashed_value = hashlib.sha256(hashlib.sha256(data_input.encode('utf-8')).digest()).hexdigest()
    return hashed_value
    
class merkle_tree:
    #Hasing individual Datas:
    #level 0
    def level_0():
        hash_tx_data_0 = []
        # hash_tx_data_0 = data_hashing()
        for i in range(0,4,1):
            temp = labels_O[i]
            hash_tx_data_0.append(temp)
        return hash_tx_data_0

    #level 1
    def level_1(b):
        hash_tx_data_1 = []  
        for i in range(0, len(b), 2):
            temp = b[i] + b[i + 1]
            temp_hash = data_hashing(temp)
            hash_tx_data_1.append(temp_hash)    
        return hash_tx_data_1

    #level 2
    def level_2(c):
        return data_hashing(c[0] + c[1])

def merkle_tree_creation():
    lvl_0 = merkle_tree.level_0()
    lvl_1 = merkle_tree.level_1(lvl_0)
    lvl_2 = merkle_tree.level_2(lvl_1)
    return lvl_0, lvl_1, lvl_2

In [68]:
def draw_tree(l_0, l_1, l_2, txd):
    G = nx.Graph()
    positions = {}
    coordinates = [
        [0, 4],
        [-2, 3],
        [2, 3],
        [-3, 2],
        [-1, 2],
        [1, 2],
        [3, 2],
        [-3, 1],
        [-1, 1],
        [1, 1],
        [3, 1]
    ]
    parents = [0, 0, 0, 1, 1, 2, 2, 3, 4, 5, 6]
    for index in range(11):
        G.add_node(index)
        G.add_edge(index, parents[index])
        positions[index] = coordinates[index]
#nx.draw(G, positions, node_size = 1000)

#nx.draw_networkx_labels(G, positions, labels = labels)
    nx.draw(G, positions, node_size = 1000, node_color='w', alpha=1, node_shape='s')
    labels = {
        0: l_2[:8]+'...',
        1: '{:.8}...'.format(l_1[0]),
        2: '{:.8}...'.format(l_1[1]),
        3: '{:.8}...'.format(l_0[0]),
        4: '{:.8}...'.format(l_0[1]),
        5: '{:.8}...'.format(l_0[2]),
        6: '{:.8}...'.format(l_0[3]),
        7: "'{}'".format('tx1'),
        8: "'{}'".format('tx2'),
        9: "'{}'".format('tx3'),
        10:"'{}'".format('tx4'),
    }
    nx.draw_networkx_labels(G, positions, labels = labels)

In [69]:
labels_a = {}
    
labels_a[0] = hashlib.sha256(hashlib.sha256(b"tx1").digest()).hexdigest()
labels_a[1] = hashlib.sha256(hashlib.sha256(b"tx2").digest()).hexdigest()
labels_a[2] = hashlib.sha256(hashlib.sha256(b"tx3").digest()).hexdigest()
labels_a[3] = hashlib.sha256(hashlib.sha256(b"tx4").digest()).hexdigest()

htxd_0, htxd_1, htxd_2 = merkle_tree_creation()

## Draw tree

In [74]:
draw_tree(htxd_0, htxd_1, htxd_2, labels_a)

2. Provide a proof of correctness of leaf tx1 and set of leafs tx1-tx2 (1 point)

In [70]:
def test_hash_tx_1_and_tx_2():
    test_set = {}
    test_set[0] = hashlib.sha256(hashlib.sha256(b"tx1").digest()).hexdigest()
    test_set[1] = hashlib.sha256(hashlib.sha256(b"tx2").digest()).hexdigest()
    test_node = test_set[0] +  test_set[1]
    temp = {}
    temp[0] =  data_hashing(test_node)
    test_node = temp
    if(test_node[0]==htxd_1[0]):   # we compare our test hash with has in vertex
        print("True")
    else:
        print("False")

In [71]:
test_hash_tx_1_and_tx_2()

True


3. Change the value on leaf tx1 and recompute corresponding hashes. Plot newly obtained Merkle hash tree (1 point)

In [72]:
labels_b = {}
    
labels_b[0] = hashlib.sha256(hashlib.sha256(b"tx1").digest()).hexdigest() + str(1)
labels_b[1] = hashlib.sha256(hashlib.sha256(b"tx2").digest()).hexdigest()
labels_b[2] = hashlib.sha256(hashlib.sha256(b"tx3").digest()).hexdigest()
labels_b[3] = hashlib.sha256(hashlib.sha256(b"tx4").digest()).hexdigest()

htxd_00,htxd_11,htxd_22 = merkle_tree_creation()

## Draw tree

In [75]:
draw_tree(htxd_00,htxd_11,htxd_22,labels_b)